In [10]:
# 라이브러리 import
import timeit
import numpy as np
import pandas as pd

# 배치 경사 하강법
def batch_gradient_descent(x : np.ndarray, y : np.ndarray, theta : np.ndarray, alpha : float, m : int, epoch : int, batchSize : int):
    xTrans = x.transpose()
    for i in range(0, epoch):
        hypothesis = np.dot(x, theta)
        loss = hypothesis - y
        cost = np.sum(loss ** 2) / (2 * m)
        gradient = np.dot(xTrans, loss) / m
        theta = theta - alpha * gradient
    return ("배치" , cost, theta)


# 확률적 경사 하강법
def stochastic_gradient_descent(x : np.ndarray, y : np.ndarray, theta : np.ndarray, alpha : float, m : int, epoch : int, batchSize : int):
    xTrans = x.transpose()
    for i in range(0, epoch):
        for j in range(0, m):
            hypothesis = np.dot(x[j:j + 1, :], theta)
            loss = hypothesis - y[j:j + 1]
            cost = np.sum(loss ** 2) / (2 * m)
            gradient = np.dot(xTrans[:, j:j + 1], loss) / m
            theta = theta - alpha * gradient
    return ("확률적" , cost, theta)

# 미니 배치 경사 하강법
def mini_batch_gradient_descent(x : np.ndarray, y : np.ndarray, theta : np.ndarray, alpha : float, m : int, epoch : int, batchSize : int):
    xTrans = x.transpose()
    for i in range(0, epoch):
        for j in range(0, m, batchSize):
            hypothesis = np.dot(x[j:j + batchSize, :], theta)
            loss = hypothesis - y[j:j + batchSize]
            cost = np.sum(loss ** 2) / (2 * m)
            gradient = np.dot(xTrans[:, j:j + batchSize], loss) / m
            theta = theta - alpha * gradient
    return ("미니 배치" , cost, theta)

# 미니 배치 경사 하강법 + 미니 배치에서 데이터 30개를 랜덤하게 뽑아서 사용
def mini_batch_random_gradient_descent(x : np.ndarray, y : np.ndarray, theta : np.ndarray, alpha : float, m : int, epoch : int, batchSize : int):
    xTrans = x.transpose()
    for i in range(0, epoch):
        for j in range(0, m, batchSize):
            # 0~99까지의 중복되지 않는 랜덤한 정수 30개 생성
            random_idx_list = np.random.choice(100, 30, replace=False)
            # 배열에서 random_idx_list 안에 있는 인덱스만 추출
            hypothesis = np.dot(x[j:j + batchSize, :][random_idx_list], theta)
            loss = hypothesis - y[j:j + batchSize][random_idx_list]
            cost = np.sum(loss ** 2) / (2 * m)
            gradient = np.dot(xTrans[:, j:j + batchSize][:, random_idx_list], loss) / m
            theta = theta - alpha * gradient
    return ("미니 배치 랜덤" , cost, theta)

# 경사하강법 속도 및 정확도 비교
def checkSpeedAndEfficiency(func_list : list, x : np.ndarray, y : np.ndarray, theta : np.ndarray, alpha : float, m : int, epoch : int, batchSize : int):
    test_list = []
    for func in func_list:
        
        start = timeit.default_timer()
        result = func(x, y, theta, alpha, m, epoch, batchSize)
        stop = timeit.default_timer()
        
        name, cost, th = result
        running_time = stop - start
        
        cost_per_running_time = cost * running_time
        print(name, f"경사 하강법 (에포크 {epoch})")
        print("시간 (running_time)                    :", format(running_time, ".15f"))
        print("비용 (cost)                            :", format(cost, ".15f"))
        print("효율 (cost * running_time)             :", format(cost_per_running_time, ".15f"))
        print()
        # test_list.append({"name": name, "time": running_time, "cost": cost, "cost_per_time": cost_per_running_time})
        test_list.append({"name": name, "cost_per_time": cost_per_running_time, running_time: running_time, cost: cost})
        test_list = sorted(test_list, key=lambda x: x["cost_per_time"])
    
    print("-"*60)
    print("효율 순서")
    for item in test_list:
        # print(f"{count}." , item["name"], ":", format(item["cost_per_time"], ".15f"))
        print( format(item["cost_per_time"], ".15f"), "(", item["name"], ")")


# ----------------------------------------------------------------------------------------------------------------------

# 함수 리스트
func_list = [batch_gradient_descent, \
            stochastic_gradient_descent, \
            mini_batch_gradient_descent, \
            mini_batch_random_gradient_descent]


# 데이터 로드
df = pd.read_csv('data/byong_data_set1.csv')

# 데이터 세팅
x = df[['height', 'weight']].values # 특성 (feature)
y = df[['bmi']].values # 라벨 (label)
m, n = np.shape(x) # m : 샘플 개수, n : 특성 개수
epoch = 5 # 에포크 (epoch)
theta = np.ones(n).reshape(-1, 1) # 임의 가중치
alpha = 0.00001 # 학습률
batchSize = 100 # 미니 배치 크기

# 체크 
checkSpeedAndEfficiency(func_list, x, y, theta, alpha, m, epoch, batchSize)




배치 경사 하강법 (에포크 419)
시간 (running_time)                    : 1.141499400022440
비용 (cost)                            : 2.547014201709169
효율 (cost * running_time)             : 2.907415183099650



KeyboardInterrupt: 